# mrbles pipeline example

## Load necessary modules

In [3]:
# Data modules
import os
import numpy as np
import pandas as pd
import xarray as xr

# For development purposes or direct source code use only. Comment out or do not load when not developing package.
import sys  # Necessary to add folders to Pyhton's PATH
sys.path.insert(0, '../')  # Folder of the cloned source code.

# Load mrbles package
import mrbles

# For standard matplotlib
import matplotlib.pyplot as plt
# Enable plotting in jupyter notebook
%matplotlib inline
# Can also be set to notebook, which makes the graphs interactive, but uses more memory
# %matplotlib notebook
import seaborn as sns  # Makes better looking plots, based on matplotlib

# For fancy interactive Plotly graphs
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot  # For plotly offline mode
init_notebook_mode(connected=True)  # Enable plotly to notebook mode

## Load image set

The dictionary keys of image folders and image patterns must match.

The instatiatiation of the object will search for the files. An error is thrown if no images are found.

In [4]:
image_folders = {"125 nM" : r"C:\DATA\Huy\20180207 CN",
                 "250 nM" : r"C:\DATA\Huy\20180207 CN",
                 "500 nM" : r"C:\DATA\Huy\20180207 CN",
                 "1000 nM" : r"C:\DATA\Huy\20180207 CN",
                 "2000 nM" : r"C:\DATA\Huy\20180207 CN",
                 "Biotin" : r"C:\DATA\Huy\20180111 CN"
}
image_patterns = {"125 nM" : r"20180207_CN_selected_125nM_([1-9]|[1-9][0-9])_MMStack_Pos0.ome.tif",
                  "250 nM" : "20180207_CN_selected_250nM_([1-9]|[1-9][0-9])_MMStack_Pos0.ome.tif",
                  "500 nM" : r"20180207_CN_selected_500nM_([1-9]|[1-9][0-9])_MMStack_Pos0.ome.tif",
                  "1000 nM" : r"20180207_CN_selected_1uM_([1-9]|[1-9][0-9])_MMStack_Pos0.ome.tif",
                  "2000 nM" : r"20180207_CN_selected_2uMb_([1-9]|[1-9][0-9])_MMStack_Pos0.ome.tif",
                  "Biotin" : r"20180111_CN_Final_Normal_biotinb_([1-9]|[1-9][0-9])_MMStack_Pos0.ome.tif"
}
mrbles_images = mrbles.Images(folders=image_folders, file_patterns=image_patterns)

No files found in 125 nM with given parameters.
No files found in 250 nM with given parameters.
No files found in 500 nM with given parameters.
No files found in 1000 nM with given parameters.
No files found in 2000 nM with given parameters.
No files found in Biotin with given parameters.


In [5]:
mrbles_images.files

{'125 nM': None,
 '250 nM': None,
 '500 nM': None,
 '1000 nM': None,
 '2000 nM': None,
 'Biotin': None}

### Load the images into memory
Remember: you need to have the memory in your comoputer to load it into the memory, otherwise it will take a very long time to analyze, since your computer will use the swap file on your hard drive. Rule of thumb: have at least 3x the memory of the size of your images

In [6]:
mrbles_images.load()

TypeError: 'NoneType' object is not iterable

### Rename channels
To make analysis easier you need to rename the Cy5 channels to be all the same

In [ ]:
mrbles_images.rename_channel('Cy5-E', 'Cy5')
mrbles_images.rename_channel('Cy5-E 10%', 'Cy5')

### Crop images
If necessary crop the images, this can be done at all time and does not require reloading of the images. This can also reduce memory usage. If you do not use the automated circle ROI finding, this is the point where you set and ROI.

In [ ]:
mrbles_images.crop_x = slice(90, 940)
mrbles_images.crop_y = slice(90, 940)

### Slicing and displaying

The mrbles package uses a custom dataframe format, which is combination of dict, which is a standard Python dataframe using keys and values (your sets, defined using the keys of the imaga folders/patterns) and Xarray, whih is used for multi-dimensional arrays. It can be slices using as a Xarray DataArray using Xarray syntax (similar to Pandas). The reason for using a custom dataframe is that Xarray (and any other array method) cannot work with 'jagged' data, which is not suitable for combining sets/conditions that have variable numbers of files.

mrbles dataframe structure:
```python
mrbles_images['set name', file number, 'channel name', y-slice, x-slice]
```

In [ ]:
plt.figure(dpi=150)
plt.axis('off')
plt.tight_layout()
plt.imshow(mrbles_images['125 nM', 5, 'Brightfield']);  # set 63 nM, file #3 (Pyhton is 0-indexed), channel 'Brightfield'

## Find MRBLEs in images

Instatiate a MBRLEs finding object, giving initial bead_size (pixels), which must be set. More fine-tune settings can be set by calling object.settings.area_max etc. Otherwise, default settings are used.

Settings can be accessed via .settings. Use Tab to see options and CRTL+Tab to see documentation.

The circle_size parameters enables automated circle ROI finding. It will find the center and then creates a circular ROI, around that center with the radius provided in pixels.

Set bead_size to an estimated bead width in pixels. This will estimate area_min and area_max, which are used for filtering objects.

In [ ]:
find_mrbles = mrbles.Find(bead_size=14, border_clear=True, circle_size=350)
find_mrbles.settings.eccen_max = 0.65
find_mrbles.settings.parallelize = True  # Experimental method, does not work well on computers with limited resources.

### Start the mrbles bead finding

In [ ]:
find_mrbles.find(mrbles_images[:, : , 'Brightfield'])

### Different masked regions
The Find method provides several different regions of the bead, namely: 

In [ ]:
find_mrbles['125 nM'].c

The mask 'mask_check' gives you an image with the found circular region and the found beads.

In [ ]:
plt.figure(dpi=450)
plt.axis('off')
plt.tight_layout()
plt.imshow(find_mrbles['125 nM', 5, 'mask_check']);

In [ ]:
plt.figure(dpi=450)
plt.axis('off')
plt.tight_layout()
plt.imshow(find_mrbles['125 nM', 5, 'mask_ring']);

## Create Reference spectra
This uses a similar setup as Images() and Find(), in which you provide folders and patterns.

In [ ]:
# Channel settings
DECODE_CHANNELS = slice('l-435','l-780')  # Channel range for decoding
OBJECT_CHANNEL = 'Brightfield'  # Channel for bead finding

# Reference files
REF_FOLDER = {
    "Dy": r"C:\DATA\20170406 - Reference files - KARA",
    "Sm": r"C:\DATA\20170406 - Reference files - KARA",
    "Tm": r"C:\DATA\20170406 - Reference files - KARA",
    "Eu": r"C:\DATA\20170406 - Reference files - KARA",
    "bkg": r"C:\DATA\Huy\20180111 CN\20180111_CN_Final_Normal_biotinb_5"
}
REF_FILES = {"Dy" : "Dy_Solos_3_MMStack_Pos0.ome.tif",
             "Sm" : "Sm_solos_5_MMStack_Pos0.ome.tif",
             "Tm" : "Tm_solos_3_MMStack_Pos0.ome.tif",
             "Eu" : "Eu_solos_6_MMStack_Pos0.ome.tif",
             "bkg" : "20180111_CN_Final_Normal_biotinb_5_MMStack_Pos0.ome.tif"
}

### Instatiating References()
Set bead_size as in Find().
crop_x/crop_y are the general (rectangular) ROI.
bkg_roi is the background region. Choose an empty spot in the 'bkg' image selected above.

In [ ]:
spec_object = mrbles.References(REF_FOLDER, REF_FILES, OBJECT_CHANNEL, DECODE_CHANNELS, bead_size=18)
spec_object.crop_x = slice(262, 762)
spec_object.crop_y = slice(262, 762)
spec_object.bkg_roi = [slice(390, 738), slice(377, 733)]

### Load References

In [ ]:
spec_object.load()

### Show plot and 'bkg' ROI
Background ROI should not show any beads.

In [ ]:
spec_object.plot()

## Spectral unmixing and get ratios
This will linearly unmix the lanthanide channels into unmixed lanthanide images, using the References() object above.

In [ ]:
ratio_images = mrbles.Ratio(spec_object)

### Unmix images and get ratios
combine_data=data is for including the data inside the Ratio() object for later ease/use. Be careful with how many channels to include, since this will consume memory rapidly (since you are copying them into memory). Always include [] around the channel name, even if it is only 1 name, otherwise it will not copy its name and throws an error!

In [ ]:
ratio_images.get(mrbles_images[:, :, DECODE_CHANNELS], 'Eu', combine_data=mrbles_images[:, :, ['Cy5']])

In [ ]:
ratio_images['125 nM'].c

## Extract data from each MRBLE

In [ ]:
extract_data = mrbles.Extract()

In [ ]:
extract_data.get(ratio_images[:, :, ['Dy_ratio', 'Sm_ratio', 'Tm_ratio', 'bkg', 'Eu', 'Cy5']], 
                 find_mrbles[:, :, ['mask_ring', 'mask_inside', 'mask_full', 'mask_bkg']])

In [ ]:
extract_data.filter(bkg_factor=2.0, ref_factor=2.0, bkg='bkg.mask_full', ref='Eu.mask_inside')

## Decode - Biotin

In [ ]:
seq_file = pd.read_excel(r'C:\DATA\Huy\20180105_SEQ_CN_Final-Match-List.xlsx')
seq_file.loc[(seq_file.code==12), ('Dy', 'Sm', 'Tm')] = [0, 0.41682, 0.02844]
seq_file.loc[(seq_file.code==13), ('Dy', 'Sm', 'Tm')] = [0, 0.47785, 0.04081]

target = seq_file.loc[(seq_file.set=='match'), ['Dy', 'Sm', 'Tm']].values
sequences = seq_file.loc[(seq_file.set=='match')].reset_index(drop=True)

target_biotin = seq_file.loc[:, ['Dy', 'Sm', 'Tm']].values
sequences_biotin = seq_file.reset_index(drop=True)

In [ ]:
biotin_set = extract_data.data.loc[('Biotin')]

In [ ]:
biotin_set

In [ ]:
mrbles_decode_biotin = mrbles.Decode(target_biotin)
mrbles_decode_biotin.settings.icp.train = False

In [ ]:
mrbles_decode_biotin.decode(biotin_set.loc[:,('Dy_ratio.mask_inside', 'Sm_ratio.mask_inside', 'Tm_ratio.mask_inside')], combine_data=biotin_set)

### Subtract local background

In [ ]:
extract_set_biotin = mrbles_decode_biotin.data.loc[(mrbles_decode_biotin.data.confidence > 0.95), ('code', 'flag', 'Cy5.mask_ring')]
extract_set_biotin.loc[:, 'Cy5.mask_ring'] -= mrbles_decode_biotin.data.loc[:, ('Cy5.mask_bkg')]

In [ ]:
extract_set_biotin

### Analyze per-code data

In [ ]:
mrbles_biotin_data = mrbles.Analyze(seq_list=sequences_biotin)

In [ ]:
mrbles_biotin_data.analyze(extract_set_biotin)

In [ ]:
mrbles_biotin_data.data

## Decode - Concentrations

In [ ]:
mrbles_decode = mrbles.Decode(target)

In [ ]:
mrbles_decode.settings.icp.train = True

In [ ]:
bead_set = extract_data.data

### Train bead-set by cycling thourgh good sets

In [ ]:
mrbles_decode.decode(bead_set.loc['250 nM', ('Dy_ratio.mask_inside', 'Sm_ratio.mask_inside', 'Tm_ratio.mask_inside')])

### Decode on whole set using trained mrbles_decode object, and combine with bead_set data

In [ ]:
mrbles_decode.decode(bead_set.loc[:, ('Dy_ratio.mask_inside', 'Sm_ratio.mask_inside', 'Tm_ratio.mask_inside')], combine_data=bead_set)

### Analyze per-code data

In [ ]:
mrbles_final = mrbles.Analyze(seq_list=sequences)

Subtract local background

In [ ]:
extract_set = mrbles_decode.data.loc[(mrbles_decode.data.confidence > 0.95), ('code', 'flag', 'Cy5.mask_ring')]
extract_set.loc[:, 'Cy5.mask_ring'] -= mrbles_decode.data.loc[:, ('Cy5.mask_bkg')]

In [ ]:
extract_set

In [ ]:
mrbles_final.analyze(extract_set)

In [ ]:
mrbles_final.data

## Normalize data using biotin data

In [ ]:
norm_data = mrbles_biotin_data.data
norm_max = norm_data['mean'].max()

In [ ]:
norm_data['mean_scaled'] = mrbles_biotin_data.data['mean'] / norm_data['mean'].max()
norm_data['median_scaled'] = mrbles_biotin_data.data['median'] / norm_data['median'].max()
norm_data['sd_scaled'] = mrbles_biotin_data.data['sd'] / norm_data['mean'].max()
norm_data['se_scaled'] = mrbles_biotin_data.data['sd_scaled'] / np.sqrt(norm_data['N'])

In [ ]:
norm_data

In [ ]:
# norm_data.to_csv("C:/Users/bjorn/Desktop/20180207 - Calibration Biotin.csv")

In [ ]:
beads_data = mrbles_final.data

In [ ]:
beads_data

In [ ]:
set_codes = np.unique(beads_data['set.code'])

In [ ]:
for code in set_codes:
    norm_mean = norm_data.loc[norm_data['set.code'] == code, 'mean_scaled'].values
    norm_sd = norm_data.loc[norm_data['set.code'] == code, 'sd_scaled'].values
    
    data_mean = beads_data.loc[beads_data['set.code'] == code, 'mean'].values
    data_median = beads_data.loc[beads_data['set.code'] == code, 'median'].values
    data_sd = beads_data.loc[beads_data['set.code'] == code, 'sd'].values
    data_n = beads_data.loc[beads_data['set.code'] == code, 'N'].values
    
    mean_norm = (data_mean / norm_mean)
    median_norm = (data_median / norm_mean)
    sd_norm = np.abs(mean_norm) * (np.sqrt((data_sd / data_mean) ** 2 + (norm_sd / norm_mean)**2))
    cv_norm = mean_norm / sd_norm
    se_norm = sd_norm / np.sqrt(data_n)
    
    beads_data.loc[beads_data['set.code'] == code, 'mean_norm'] = mean_norm
    beads_data.loc[beads_data['set.code'] == code, 'median_norm'] = median_norm
    beads_data.loc[beads_data['set.code'] == code, 'sd_norm'] = sd_norm
    beads_data.loc[beads_data['set.code'] == code, 'cv_norm'] = cv_norm
    beads_data.loc[beads_data['set.code'] == code, 'se_norm'] = se_norm

In [ ]:
beads_data

In [ ]:
# beads_data.to_csv(r'20180213 - Per-Code data.csv')

In [ ]:
bead_set1 = mrbles_decode.data

confidence = 0.95

colors = np.multiply(bead_set1[(bead_set1.confidence > confidence)].code.values.astype(int), np.ceil(255/len(target)))

bead_ratios = go.Scatter3d(
    name='Bead ratios',
    x=bead_set1.loc[(bead_set1.confidence > confidence), ('Dy_ratio.mask_inside_icp')].values,
    y=bead_set1.loc[(bead_set1.confidence > confidence), ('Sm_ratio.mask_inside_icp')].values,
    z=bead_set1.loc[(bead_set1.confidence > confidence), ('Tm_ratio.mask_inside_icp')].values,
    text=bead_set1.loc[(bead_set1.confidence > confidence), ('code')].values + 1,
    mode='markers',
    marker=dict(
        size=3,
        color=colors, 
        colorscale='Rainbow',
        opacity=0.6
    )
)

target_ratios = go.Scatter3d(
    name='Target ratios',
    x=target[:,0],
    y=target[:,1],
    z=target[:,2],
    text=list(range(1, len(target)+1)),
    mode='markers',
    marker=dict(
        size=4,
        color='black',
        symbol="diamond"
    )
)

mean_ratios = go.Scatter3d(
    name='GMM mean ratios',
    x=mrbles_decode.settings.gmm.means[:,0],
    y=mrbles_decode.settings.gmm.means[:,1],
    z=mrbles_decode.settings.gmm.means[:,2],
    text=list(range(1, len(target)+1)),
    mode='markers',
    marker=dict(
        size=4,
        color='red',
        opacity=0.5,
        symbol="diamond"
    )
)

data = [bead_ratios, target_ratios, mean_ratios]
layout = go.Layout(
    showlegend=True,
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [ ]:
bead_set = mrbles_decode_biotin.data

confidence = 0.95

colors = np.multiply(bead_set[(bead_set.confidence > confidence)].code.values.astype(int), np.ceil(255/len(target_biotin)))

bead_ratios = go.Scatter3d(
    name='Bead ratios',
    x=bead_set.loc[(bead_set.confidence > confidence), ('Dy_ratio.mask_inside_icp')].values,
    y=bead_set.loc[(bead_set.confidence > confidence), ('Sm_ratio.mask_inside_icp')].values,
    z=bead_set.loc[(bead_set.confidence > confidence), ('Tm_ratio.mask_inside_icp')].values,
    text=bead_set.loc[(bead_set.confidence > confidence), ('code')].values + 1,
    mode='markers',
    marker=dict(
        size=3,
        color=colors, 
        colorscale='Rainbow',
        opacity=0.6
    )
)

target_ratios = go.Scatter3d(
    name='Target ratios',
    x=target_biotin[:,0],
    y=target_biotin[:,1],
    z=target_biotin[:,2],
    text=list(range(1, len(target_biotin)+1)),
    mode='markers',
    marker=dict(
        size=4,
        color='black',
        symbol="diamond"
    )
)

mean_ratios = go.Scatter3d(
    name='GMM mean ratios',
    x=mrbles_decode_biotin.settings.gmm.means[:,0],
    y=mrbles_decode_biotin.settings.gmm.means[:,1],
    z=mrbles_decode_biotin.settings.gmm.means[:,2],
    text=list(range(1, len(target)+1)),
    mode='markers',
    marker=dict(
        size=4,
        color='red',
        opacity=0.5,
        symbol="diamond"
    )
)

data = [bead_ratios, target_ratios, mean_ratios]
layout = go.Layout(
    showlegend=True,
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)

# More analysis

In [ ]:
bead_set  # Per-bead data

In [ ]:
plt.figure(dpi=100)
bead_set.loc[:, 'Cy5.mask_inside'].hist(bins=100);

In [ ]:
plt.figure(dpi=100)
cy5_min_bkg = (bead_set.loc[:, 'Cy5.mask_inside'] - bead_set.loc[:, 'Cy5.mask_bkg']).dropna()
plt.hist(cy5_min_bkg, bins=100);

In [ ]:
plt.figure(dpi=100)
sns.distplot(cy5_min_bkg, bins=100);

In [ ]:
extract_set_biotin.loc[:, 'Cy5.mask_ring'].plot.kde();

In [ ]:
# Initialize the FacetGrid object
pal = sns.cubehelix_palette(len(bead_set.code.unique()), rot=-.25, light=.7)
g = sns.FacetGrid(extract_set_biotin, row="code", hue="code", aspect=15, size=.5, palette=pal)

# Draw the densities in a few steps
g.map(sns.kdeplot, "Cy5.mask_ring", clip_on=False, shade=True, alpha=1, lw=1.5, bw=.2)
g.map(sns.kdeplot, "Cy5.mask_ring", clip_on=False, color="w", lw=2, bw=.2)
g.map(plt.axhline, y=0, lw=2, clip_on=False)

# Define and use a simple function to label the plot in axes coordinates
def label(x, color, label):
    ax = plt.gca()
    ax.text(0, .2, label, fontweight="bold", color=color, 
            ha="left", va="center", transform=ax.transAxes)

g.map(label, "Cy5.mask_ring")

# Set the subplots to overlap
g.fig.subplots_adjust(hspace=-.25)

# Remove axes details that don't play will with overlap
g.set_titles("")
g.set(yticks=[])
g.despine(bottom=True, left=True)